## useful links, threads
- https://www.tetranyde.com/blog/embedding-superset
- https://github.com/apache/superset/issues/17206
- https://github.com/apache/superset/issues/16398
- https://github.com/apache/superset/issues/10354#issuecomment-758543077
- https://github.com/metriql/metriql-superset/blob/main/metriql2superset/superset.py#L21

In [ ]:
import json
import requests

In [ ]:
base_url = "http://localhost:10088"

In [ ]:
def login():
    # Login to Superset to get an access token
    payload = json.dumps({
        "username": "<>",
        "password": "<>",
        "provider": "db",
        "refresh": "true"
    })
    headers = {
        'Content-Type': 'application/json'
    }
    response = requests.post(f"{base_url}/api/v1/security/login",
                             headers=headers, data=payload)
    response.raise_for_status()
    return response.json()['access_token']

In [ ]:
# Request guest token from Superset
guest_token_payload = {
    "user": {
        "username": '<>',
        "first_name": '<>',
        "last_name": '<>',
    },
    "resources": [
        {
            "type": "dashboard",
            "id": "<>",
        }
    ],
    # Include Row Level Security (RLS) rules if applicable
    "rls": []
}

In [ ]:
def get_guest_token():
    session = requests.Session()
    session.headers['Authorization'] = f"Bearer {access_token}"
    session.headers['Content-Type'] = "application/json"
    
    csrf_url = f"{base_url}/api/v1/security/csrf_token"
    csrf_response = session.get(csrf_url, verify=False)
    csrf_response.raise_for_status()
    csrf_token = csrf_response.json()['result']
    
    session.headers['Referer'] = csrf_url
    session.headers['X-CSRFToken'] = csrf_token
    
    response = session.post(f"{base_url}/api/v1/security/guest_token", 
                        data=json.dumps(guest_token_payload), verify=False)
    
    return response

resp = get_guest_token()